In [1]:
import pandas as pd
import numpy as np
from scipy.stats import boxcox
from sklearn.preprocessing import StandardScaler

In [35]:
df = pd.read_csv('./merged_file.csv')
scalar = StandardScaler()

def boxcox_tscore(df, column_name): # boxcox 변환 후 t점수를 계산하는 함수
    df[column_name] = df[column_name] + 1e-20 # 음수이거나 0이면 변환이 안되기 때문에 아주 작은 양수 값을 더해줌
    df[f'{column_name}_boxcox'], fitted_lambda = boxcox(df[column_name]) # Box-cox 변환, fitted_lambda: Box-cox 변환 과정에서 자동으로 최적화된 lambda값, 데이터의 분포를 정규에 가깝게 만들어줌, 이 코드에서는 쓰이지 않았지만, 변환된 데이터를 원래의 스케일로 복원할 때 씀 
    df[f'{column_name}_scaled'] = scalar.fit_transform(df[[f'{column_name}_boxcox']]) # 정규화를 위한 평균, 표준편차 계산
    if (column_name == 'crimeRate'):
        df[f'{column_name}_tscore'] = df[f'{column_name}_scaled'] *-1 *10 + 50 # 범죄율의 경우 낮아야 좋으므로 -1를 곱해줌
    else:
         df[f'{column_name}_tscore'] = df[f'{column_name}_scaled'] *10 + 50 # t점수 계산

boxcox_tscore(df, "libraryCount")
boxcox_tscore(df, "busStation")
boxcox_tscore(df, "cultureCount")
boxcox_tscore(df, "medicalCount")
boxcox_tscore(df, "crimeRate")
boxcox_tscore(df, "youthRate")
boxcox_tscore(df, "supermarket")
boxcox_tscore(df, "parkRate")
boxcox_tscore(df, "academyCount")

In [36]:
# t점수가 제대로 나왔는지 비교하기 쉽게 column을 배치
df1 = df[['gu', 'dong', 'libraryCount','libraryCount_tscore','busStation','busStation_tscore','cultureCount','cultureCount_tscore','medicalCount','medicalCount_tscore','crimeRate','crimeRate_tscore','youthRate','youthRate_tscore','supermarket','supermarket_tscore','parkRate','parkRate_tscore','academyCount','academyCount_tscore']]

In [37]:
# 카테고리 별로 항목이 두 개인 것은 0.5를 곱해서 점수를 하나로 만들어주고, DB Schema 이름에 맞게 column명 변경
df2 = df1.copy()
df2['eduScore'] = (df2['libraryCount_tscore']) * 0.5 + (df2['academyCount_tscore'] * 0.5)
df2['envScore'] = df2['parkRate_tscore']
df2['safeScore'] = df2['crimeRate_tscore']
df2['welScore'] = (df2['cultureCount_tscore']) * 0.5 + (df2['medicalCount_tscore'] * 0.5)
df2['popScore'] = df2['youthRate_tscore']
df2['transScore'] = df2['busStation_tscore']
df2['convScore'] = df2['supermarket_tscore']


In [40]:
df3 = df2[['gu', 'dong','eduScore','envScore','safeScore','welScore','popScore','transScore','convScore']]
df3.to_csv('./boxcox_tscore_result.csv', encoding='utf-8-sig')